<a href="https://colab.research.google.com/github/GinkGoPi/researches/blob/main/token-stake/simple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simeple about token staking and reward

> 模拟简单模型的staking和reward的逻辑和计算

`Staking`中用户存入了一定数量的token，pool中reward token通过外部打入，然后在一个周期内将其按照用户质押的数量占比分配掉reward token

合约中奖励的分配是**平均每秒**的释放方式，比如
```
一个周期为60个单位时间，打入奖励token的数量6,此时：
lastUpdateTime=0；periodFinish=60；rewardPerTokenStored=0
rewardRate = 6/60 = 0.1
rewards[UserA] = 0

UserA在第0个单位时间stake了100个token,此时：
    balanceOf(UserA) = 100
    total = 100

{ 
    第10个单位时间，此时计算UserA的收益：
    temp_rewardPerTokenStored = rewardPerTokenStored + (lastTimeRewardApplicable - lastUpdateTime) * rewardRate / total = 0 + (10-0)*0.1/100 = 0.001
    rewards = rewards[UserA] + balanceOf(UserA) * (temp_rewardPerTokenStored - userRewardPerTokenPaid[UserA]) = 0 + 100*(0.001-0)=1
}

UserB在第10个单位时间stake了200个token，此时：
    rewardPerTokenStored = rewardPerTokenStored + (lastTimeRewardApplicable - lastUpdateTime) * rewardRate / total = 0 + (10-0)*0.1/100 = 0.01
    lastUpdateTime = 10
    userRewardPerTokenPaid[UserB] = rewardPerTokenStored = 0.01
    balanceOf(UserB) = 200
    total = 300

{ 
    第20个单位时间，此时计算UserA的收益：
    temp_rewardPerTokenStored = rewardPerTokenStored + (lastTimeRewardApplicable - lastUpdateTime) * rewardRate / total = 0.01 + (20-10)*0.1/300 = 0.0133
    rewards = rewards[UserA] + balanceOf(UserA) * (temp_rewardPerTokenStored - userRewardPerTokenPaid[UserA]) = 0 + 100*(0.0133-0)=1.33

    第20个单位时间，此时计算UserB的收益：
    temp_rewardPerTokenStored = 0.0133
    rewards = rewards[UserB] + balanceOf(UserB) * (temp_rewardPerTokenStored - userRewardPerTokenPaid[UserB]) = 0 + 200*(0.0133-0.01) = 0.66
}
    
```

In [1]:
import calendar
import time

In [2]:
calendar.timegm(time.gmtime())

1675318795

In [3]:
class SimepleStakingRewards(object):
    
    def __init__(self) -> None:
        self.stakingToken = 'stakingToken'
        self.rewardsToken = 'rewardsToken'

        self.periodFinish = 0
        self.lastUpdateTime = 0
        self.duration = 1 * 60*60*24 # seconds

        self.rewardRate = 0
        self.rewardPerTokenStored = 0

        self.userRewardPerTokenPaid = {'address_0': 0, 'alice': 0, 'bob': 0, 'kite': 0}
        self.rewards = {'address_0': 0, 'alice': 0, 'bob': 0, 'kite': 0}

        self.totalSupply = 0
        self.balanceOf = {'address_0': 0, 'alice': 0, 'bob': 0, 'kite': 0}
    
    def stake(self, address: str, amount: int):
        self.updateReward(address)
        self.balanceOf[address] += amount
        self.totalSupply += amount
    
    def withdraw(self, to: str, amount: int):
        self.updateReward(to)
        self.balanceOf[to] -= amount
        self.totalSupply -= amount
        return amount
      
    def updateReward(self, _account: str):
        self.rewardPerTokenStored = self.rewardPerToken()
        self.lastUpdateTime = self.lastTimeRewardApplicable()

        if _account != "address_0":
            self.rewards[_account] = self.earned(_account)
            self.userRewardPerTokenPaid[_account] = self.rewardPerTokenStored
    
    def rewardPerToken(self):
        if self.totalSupply == 0:
            return self.rewardPerTokenStored
        else:
            return self.rewardPerTokenStored + (self.rewardRate * (self.lastTimeRewardApplicable() - self.lastUpdateTime) * 1e18) / self.totalSupply

    def earned(self, _account: str):
        return ((self.balanceOf[_account] *
                (self.rewardPerToken() - self.userRewardPerTokenPaid[_account])) / 1e18) + self.rewards[_account]

    def getReward(self, to: str):
        """Gain reward token"""
        self.updateReward(to)
        reward = self.rewards[to]
        if reward > 0:
            self.rewards[to] = 0
            # rewardsToken.transfer(to, reward)
        
        return reward

    def notifyRewardAmount(self, _amount: int):
        self.updateReward("address_0")
        if self.blockTimestamp() >= self.periodFinish:
            self.rewardRate = _amount / self.duration
        else:
            remainingRewards = (self.periodFinish - self.blockTimestamp()) * self.rewardRate
            self.rewardRate = (_amount + remainingRewards) / self.duration
        

        assert(self.rewardRate > 0, "reward rate = 0")

        self.lastUpdateTime = self.blockTimestamp()
        self.periodFinish = self.blockTimestamp() + self.duration
        
    
    def lastTimeRewardApplicable(self):
        return self.periodFinish if self.periodFinish <= self.blockTimestamp() else self.blockTimestamp()

    def blockTimestamp(self):
        """Akin to `block.timestamp`"""
        return calendar.timegm(time.gmtime())



<>:68: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:68: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<ipython-input-3-8bff1e5d3881>:68: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(self.rewardRate > 0, "reward rate = 0")


In [4]:
# init
pool = SimepleStakingRewards()

print("init stake total amount", pool.totalSupply)
print("init stake rewardRate", pool.rewardRate)

init stake total amount 0
init stake rewardRate 0


In [5]:
# add new rewards
rewards = 10e18

pool.notifyRewardAmount(rewards)

print("current total amount", pool.totalSupply)
print("current rewardRate", pool.rewardRate)
print("current periodFinish", pool.periodFinish)
print("current lastUpdateTime", pool.lastUpdateTime)
print("current rewardPerTokenStored", pool.rewardPerTokenStored)

current total amount 0
current rewardRate 115740740740740.73
current periodFinish 1675408116
current lastUpdateTime 1675321716
current rewardPerTokenStored 0


In [6]:
# Alice staking

alice = "alice"
stakingAmount = 100e18

pool.stake(alice, stakingAmount)

print("current total amount", pool.totalSupply)
print("current rewardRate", pool.rewardRate)
print("current periodFinish", pool.periodFinish)
print("current lastUpdateTime", pool.lastUpdateTime)

print("current rewardPerTokenStored", pool.rewardPerTokenStored)

print("alice staked", pool.balanceOf[alice])
print("alice rewards", pool.rewards[alice])
print("alice userRewardPerTokenPaid", pool.userRewardPerTokenPaid[alice])


current total amount 1e+20
current rewardRate 115740740740740.73
current periodFinish 1675408116
current lastUpdateTime 1675321723
current rewardPerTokenStored 0
alice staked 1e+20
alice rewards 0.0
alice userRewardPerTokenPaid 0


In [7]:
print("T1 query alice current earned", pool.earned(alice))

T1 query alice current earned 347222222222222.2


In [8]:
print("T2 query alice current earned", pool.earned(alice))

T2 query alice current earned 694444444444444.4


In [9]:
# Bob staking

bob = "bob"
stakingAmount = 200e18

pool.stake(bob, stakingAmount)

print("current total amount", pool.totalSupply)
print("current rewardRate", pool.rewardRate)
print("current periodFinish", pool.periodFinish)
print("current lastUpdateTime", pool.lastUpdateTime)

print("current rewardPerTokenStored", pool.rewardPerTokenStored)

print("bob staked", pool.balanceOf[bob])
print("bob rewards", pool.rewards[bob])
print("bob userRewardPerTokenPaid", pool.userRewardPerTokenPaid[bob])

current total amount 3e+20
current rewardRate 115740740740740.73
current periodFinish 1675408116
current lastUpdateTime 1675321732
current rewardPerTokenStored 10416666666666.666
bob staked 2e+20
bob rewards 0.0
bob userRewardPerTokenPaid 10416666666666.666


In [10]:
print("T1 query bob current earned", pool.earned(bob))


T1 query bob current earned 231481481481481.66


In [11]:
print("T2 query bob current earned", pool.earned(bob))


T2 query bob current earned 385802469135802.3


In [12]:
# Bob staking again

bob = "bob"
stakingAmount = 100e18

pool.stake(bob, stakingAmount)

print("current total amount", pool.totalSupply)
print("current rewardRate", pool.rewardRate)
print("current periodFinish", pool.periodFinish)
print("current lastUpdateTime", pool.lastUpdateTime)

print("current rewardPerTokenStored", pool.rewardPerTokenStored)

print("bob staked", pool.balanceOf[bob])
print("bob rewards", pool.rewards[bob])
print("bob userRewardPerTokenPaid", pool.userRewardPerTokenPaid[bob])

current total amount 4e+20
current rewardRate 115740740740740.73
current periodFinish 1675408116
current lastUpdateTime 1675321740
current rewardPerTokenStored 13503086419753.086
bob staked 3e+20
bob rewards 617283950617284.0
bob userRewardPerTokenPaid 13503086419753.086


In [13]:
print("T3 query bob current earned", pool.earned(bob))


T3 query bob current earned 877700617283950.4


In [14]:
print("T4 query bob current earned", pool.earned(bob))


T4 query bob current earned 1051311728395062.0


In [15]:
# alice withdraw and getReward

unstakingAmount = 50e18
unstakedAmount = pool.withdraw(alice, unstakingAmount)

print('alice unstaked amount', unstakedAmount)

print("current total amount", pool.totalSupply)
print("current rewardRate", pool.rewardRate)
print("current periodFinish", pool.periodFinish)
print("current lastUpdateTime", pool.lastUpdateTime)

print("current rewardPerTokenStored", pool.rewardPerTokenStored)

alice unstaked amount 5e+19
current total amount 3.5e+20
current rewardRate 115740740740740.73
current periodFinish 1675408116
current lastUpdateTime 1675321749
current rewardPerTokenStored 16107253086419.752


In [16]:
print("alice staked", pool.balanceOf[alice])
print("alice rewards", pool.rewards[alice])
print("alice userRewardPerTokenPaid", pool.userRewardPerTokenPaid[alice])

print("Tn query alice current earned", pool.earned(alice))

alice staked 5e+19
alice rewards 1610725308641975.0
alice userRewardPerTokenPaid 16107253086419.752
Tn query alice current earned 1660328483245149.5


In [17]:
# alice getReward

aliceRewards = pool.getReward(alice)

print('alice getReward amount', aliceRewards)


alice getReward amount 1743000440917107.2


In [18]:
print("current total amount", pool.totalSupply)
print("current rewardRate", pool.rewardRate)
print("current periodFinish", pool.periodFinish)
print("current lastUpdateTime", pool.lastUpdateTime)

print("alice staked", pool.balanceOf[alice])
print("alice rewards", pool.rewards[alice])
print("alice userRewardPerTokenPaid", pool.userRewardPerTokenPaid[alice])

print("Tn query alice current earned", pool.earned(alice))

current total amount 3.5e+20
current rewardRate 115740740740740.73
current periodFinish 1675408116
current lastUpdateTime 1675321757
alice staked 5e+19
alice rewards 0
alice userRewardPerTokenPaid 18752755731922.4
Tn query alice current earned 82671957671957.61
